<h1>Langevin algorithms for Markovian Neural Networks and Deep Stochastic control</h1>

We show how to:
<ol>
    <li>Use Langevin optimizers and Layer Langevin optimizers for any Tensorflow model training</li>
    <li>Use our framework for comparing differents optimizers on a same Stochastic Optimal Control problem</li>
    <li>Use our framework to create a new Stochastic Optimal Control problem.</li>
</ol>

In [ ]:
import tensorflow as tf

<h2>1) Langevin Optimizers</h2>

Optimizers in the <tt>optimizers</tt> directory can be directly used as instances of the TensorFlow <tt>Optimizer</tt> base class.

In [ ]:
from optimizers.ladam import LAdam, LayerLAdam
from optimizers.lrmsprop import LRMSprop, LayerLRMSprop
from optimizers.ladadelta import LAdadelta, LayerLAdadelta

optimizer = LAdam(learning_rate=1e-3, sigma=1e-3)

Schedules from <tt>tf.keras.optimizers.schedules</tt> may be passed to the arguments <tt>learning_rate</tt> and to <tt>sigma</tt>.

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[100], values=[1e-3,1e-4])
sigma_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=[100], values=[1e-3,0.])

optimizer = LAdam(learning_rate=lr_schedule, sigma=sigma_schedule)

<b>Layer Langevin optimizers</b>

The argument <tt>langevin_layers</tt> specify the layers of the model that are trained with Langevin noise.

When using Layer Langevin optimizers, the function <tt>set_langevin(model)</tt> must be used after the <tt>model</tt> is compiled with this optimizer and built.

In [ ]:
from optimizers.base import set_langevin

optimizer = LayerLAdam(learning_rate=1e-3, sigma=1e-3, langevin_layers=[0,1])

model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer=optimizer, loss=tf.keras.losses.MeanSquaredError())
model(tf.random.normal((1,5))) # build the model
set_langevin(model)

<h2>2) Running an experiment for Stochastic Optimal Control</h2>

<b>Model Builder:</b>

A <tt>ModelBuilder</tt> is required for the <tt>experiment</tt>.
We provide three different <tt>ModelBuilder</tt>s: <tt>Fishing</tt>, <tt>DeepHedging</tt>, <tt>OilDrilling</tt>. We refer to the corresponding simulation files for the instantation of each of these classes.

Note that <tt>OilDrilling</tt> only works with <tt>dim=1</tt>.

In [ ]:
from models.fishing import Fishing
from models.deep_hedging import DeepHedging
from models.oil_drilling import OilDrilling

dim=5; alpha=0.9
model_builder = DeepHedging(
    T=1., N_euler=50, dim=dim,
    multiple_ctrls=False, ctrl_hidden_units=[32,32],
    ell=lambda x:(1./(1.-alpha))*tf.keras.activations.relu(x),
    a = 1.*tf.ones((dim,)), b = 0.04*tf.ones((dim,)),
    sigma = 2.*tf.ones((dim,)), rho = -0.7*tf.ones((dim,)),
    K = 1.*tf.ones((dim,)), T_COST=5e-4)

<b>Dataloader:</b>

Define the input (initial condition) of the Stochastic Control problem. We provide <tt>DatasetLoader</tt>s: <tt>ConstantLoader</tt>, <tt>ConstantMultipleLoader</tt>, <tt>DataLoaderFromMap</tt>. The arguments are:

<ul>
    <li><tt>N_train</tt>: number of trajectories for each epoch</li>
    <li><tt>N_test</tt>: number of trajectories to estimate the loss with at the end of each epoch</li>
    <li><tt>batch_size</tt></li>
    <li><tt>X0</tt>: (only for <tt>ConstantLoader</tt>): initial condition, given as a tensorflow tensor or a numpy array</li>
    <li><tt>X0_list</tt>: (only for <tt>ConstantMultipleLoader</tt>): list of initial conditions when the model as multiple inputs, given as a list of tensorflow tensors or numpy arrays</li>
    <li><tt>get_X0</tt>: (only for <tt>DataLoaderFromMap</tt>): function that takes one argument as a tensorflow tensor and that returns a tensorflow tensor with same shape</li>
</ul>

In [ ]:
from data.constant import ConstantLoader, ConstantMultipleLoader
from data.from_map import DataLoaderFromMap

s0, v0 = 1., 0.1
dataloader = ConstantMultipleLoader(
    X0_list = [s0*tf.ones((dim,)), v0*tf.ones((dim,))],
    N_train = 512*5,
    N_test = 512*25,
    batch_size = 512)

Then build the <tt>experiment</tt> with additional arguments:
<ul>
    <li><tt>optimizers</tt>: list of tensorflow optimizers to compare</li>
    <li><tt>base</tt>: path to save the results</li>
<ul>

In [ ]:
from experiment import Experiment

experiment = Experiment(
    model_builder=model_builder,
    dataloader=dataloader,
    EPOCHS=20,
    optimizers=[LAdam(learning_rate=1e-3, sigma=1e-3),
                LAdam(learning_rate=1e-3, sigma=0.)],
    base='./')

<ul>
    <li><tt>experiment.load_data()</tt>: load the data according to the dataloader; must be used before any training</li>
    <li><tt>experiment.run_experiment()</tt>: train the model for each optimizer</li>
    <li><tt>experiment.plot()</tt>: plot the training curves for each optimizer</li>
    <li><tt>experiment.plot_traj(opt_index)</tt>: plot a random trajectory (after the model is trained); <tt>opt_index</tt> is the index of the optimizer to use</li>
    <li><tt>experiment.save_data(dir)</tt>: save the training curves as <tt>csv</tt> files in the <tt>experiment.base/dir</tt> directory (the directory is created)</li>
    <li><tt>experiment.save_traj(dir)</tt>: save the random trajectory as a <tt>csv</tt> file in the <tt>experiment.base/dir</tt> directory (the directory is created)</li>
</ul>

In [ ]:
experiment.load_data()
experiment.run_experiment()
experiment.plot()

experiment.plot_traj(opt_index=1)

experiment.save_data('deep_hedging_adam')
experiment.save_traj('deep_hedging_traj')

<h2>3) Setting custom <tt>ModelBuilder</tt>s for Stochastic Optimal Control</h2>

A new <tt>ModelBuilder</tt> must be implemented as a subclass of <tt>models.base.ModelBuilder</tt> and must have a method <tt>getModel(self)</tt> which returns a Tensorflow model. The output of the model must be the (scalar) loss <tt>J</tt> of shape <tt>[batch_size, 1]</tt> or <tt>[batch_size, R]</tt> where the <tt>R-1</tt> last entries will be only used for plotting purposes in <tt>experiment.plot_traj</tt>.

We refer to the source files <tt>models.fishing</tt>, <tt>models.deep_hedging</tt>, <tt>models.oil_drilling</tt>.